In [1]:
import numpy
import pandas
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import sklearn.linear_model as lm
from nltk.corpus import stopwords
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import pipeline,metrics, grid_search 
from numpy import genfromtxt

In [18]:
import random
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

with open('sarcasm_dataset.txt','r') as fname:
    file_content = fname.readlines()
random.shuffle(file_content)
output = 'tweets' + "\t" + 'label' + "\n"
for tweet_content in file_content:    
    tweet, label = tweet_content[:-3], tweet_content[-3]
    output += tweet + "\t" + label + "\n"
    
outputfile = open('dataset_csv.csv',"w")
outputfile.write(output)
outputfile.close()  


In [2]:
df = pandas.DataFrame()
df = pandas.read_csv("dataset_csv.csv", header=0, sep='\t')
df.head()

,Unnamed: 0,label,Blob Polarity,Blob Subjectivity,Capitalization,Negative Sentiment,POS_1,POS_2,POS_3,POS_4,...,first half Blob Subjectivity,first half sentiment,negative Sentiment first half,negative Sentiment second half,positive Sentiment first half,positive Sentiment second half,second half Blob Polarity,second half Blob Subjectivity,second half sentiment,sentiment
0,0,1,0.50000,0.6,0,0.031250,2.0,0.0,2.0,0.0,...,0.6,0.593750,0.031250,0.000000,0.625000,0.0000,0.0,0.0,0.000000,0.593750
1,1,1,-0.80000,0.9,0,1.444444,3.0,0.0,3.0,1.0,...,0.9,-0.638889,0.819444,0.625000,0.180556,0.1250,0.0,0.0,-0.500000,-1.138889
2,2,0,-0.50000,1.0,0,0.725000,3.0,1.0,2.0,2.0,...,1.0,-0.600000,0.725000,0.000000,0.125000,0.0000,0.0,0.0,0.000000,-0.600000
3,3,0,-0.80000,0.9,1,0.778409,1.0,0.0,7.0,3.0,...,0.9,-0.541667,0.625000,0.153409,0.083333,0.0625,0.0,0.0,-0.090909,-0.632576
4,4,1,0.78125,0.6,1,0.039062,4.0,0.0,2.0,1.0,...,0.6,0.601562,0.039062,0.000000,0.640625,0.0000,0.0,0.0,0.000000,0.601562


In [49]:
train = pandas.DataFrame()
test = pandas.DataFrame()

train = df[0:1500]
test = df[1501:]

In [36]:
tfidf_vec=TfidfVectorizer(analyzer="word",max_features=None,strip_accents='unicode',token_pattern=r'\w{1,}',lowercase=True,ngram_range=(1,3),min_df=2,use_idf=True,smooth_idf=True,norm="l2",sublinear_tf=True)

In [37]:
x_train,x_test,y_train,y_test=train_test_split(train['tweets'],train['label'],test_size=0.4,random_state=2,stratify=train['label'])

In [38]:
x_train=tfidf_vec.fit_transform(x_train)
x_test=tfidf_vec.transform(x_test)
x_train=x_train.toarray()
x_test=x_test.toarray()

In [46]:
outputfile = open('dataset_train1.txt',"w")
outputfile.write(x_train)
outputfile.close() 

In [39]:
linear_mod = lm.LogisticRegression(C=0.30,penalty="l1")

In [40]:
linear_mod.fit(x_train, y_train)

LogisticRegression(C=0.3, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [41]:
pred = list(linear_mod.predict(x_test))
target = list(y_test)


In [44]:
from sklearn.metrics import f1_score,precision_score

print f1_score(y_test,linear_mod.predict(x_test))   
print f1_score(pred, target)     

0.924428822496
0.924428822496


In [33]:
svd = TruncatedSVD(n_components=18,n_iter=9,algorithm='randomized', random_state=None, tol=0.0)
svd.fit(x_train)
x_train=svd.transform(x_train)
x_test=svd.transform(x_test)
scl = StandardScaler()
x_train=scl.fit_transform(x_train)
x_test=scl.transform(x_test)

In [55]:
tfidf_vec = TfidfVectorizer(
            analyzer="word",max_features=None,
            token_pattern=r'\w{1,}',strip_accents='unicode',
            lowercase=True,ngram_range=(1,3),
            min_df=2,use_idf=True,
            smooth_idf=True,norm="l2",
            sublinear_tf=True)

x_train,x_test,y_train,y_test=train_test_split(train['tweets'],train['label'],test_size=0.4,random_state=2,stratify=train['label'])

In [57]:
train_tfidf_matrix=tfidf_vec.fit_transform(x_train)
test_tfidf_matrix=tfidf_vec.fit_transform(x_test)

train = pandas.DataFrame(train_tfidf_matrix.toarray())
test = pandas.DataFrame(test_tfidf_matrix.toarray())

In [ ]:
svd = TruncatedSVD(algorithm="randomized", random_state=None, tol=0.0)
scl = StandardScaler()
lr = lm.LogisticRegression(class_weight="balanced", tol = 0.0001)
clf = pipeline.Pipeline([('svd', svd),
    						 ('scl', scl),
                    	     ('lr', lr)])

param_grid = {'svd__n_components' : [200,250,300,350,400],
                 'svd__n_iter':[3,4,5],
                 'lr__C': [10,11,12,13,14,15,16,17],
                  'lr__penalty':["l1","l2"]}

f_scorer = metrics.make_scorer(f1_score, greater_is_better = True)

model = grid_search.GridSearchCV(estimator = clf, param_grid=param_grid, scoring=f_scorer,
                                     verbose=10, n_jobs=-1, iid=True, refit=True, cv=10)



In [ ]:
model.fit(train, y_train)
print("Best score: %0.3f" % model.best_score_)

Fitting 3 folds for each of 240 candidates, totalling 720 fits
[CV] svd__n_iter=3, lr__penalty=l1, lr__C=10, svd__n_components=200 ..


/home/surajr/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/surajr/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/surajr/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view 

[CV] svd__n_iter=3, lr__penalty=l1, lr__C=10, svd__n_components=200 ..
[CV] svd__n_iter=3, lr__penalty=l1, lr__C=10, svd__n_components=200 ..
[CV] svd__n_iter=4, lr__penalty=l1, lr__C=10, svd__n_components=200 ..


/home/surajr/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/surajr/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)
/home/surajr/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view 

[CV]  svd__n_iter=4, lr__penalty=l1, lr__C=10, svd__n_components=200, score=0.867384 -   5.8s
[CV]  svd__n_iter=3, lr__penalty=l1, lr__C=10, svd__n_components=200, score=0.896057 -   5.9s
[CV]  svd__n_iter=3, lr__penalty=l1, lr__C=10, svd__n_components=200, score=0.922034 -   6.0s
[CV]  svd__n_iter=3, lr__penalty=l1, lr__C=10, svd__n_components=200, score=0.867384 -   6.1s
[CV] svd__n_iter=4, lr__penalty=l1, lr__C=10, svd__n_components=200 ..
[CV] svd__n_iter=5, lr__penalty=l1, lr__C=10, svd__n_components=200 ..
[CV] svd__n_iter=4, lr__penalty=l1, lr__C=10, svd__n_components=200 ..
[CV] svd__n_iter=5, lr__penalty=l1, lr__C=10, svd__n_components=200 ..


...

/home/surajr/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] svd__n_iter=4, lr__penalty=l2, lr__C=10, svd__n_components=200 ..
[CV]  svd__n_iter=5, lr__penalty=l1, lr__C=10, svd__n_components=400, score=0.909747 -  11.4s
[CV] svd__n_iter=5, lr__penalty=l2, lr__C=10, svd__n_components=200 ..


/home/surajr/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/hashing.py:197: DeprecationWarning: Changing the shape of non-C contiguous array by
descriptor assignment is deprecated. To maintain
the Fortran contiguity of a multidimensional Fortran
array, use 'a.T.view(...).T' instead
  obj_bytes_view = obj.view(self.np.uint8)


[CV] svd__n_iter=5, lr__penalty=l2, lr__C=10, svd__n_components=200 ..


Process PoolWorker-12:
Process PoolWorker-11:
Process PoolWorker-10:
Process PoolWorker-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/surajr/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/surajr/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/surajr/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
  File "/home/surajr/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/surajr/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/home/surajr/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/surajr/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._tar

In [3]:
import logging
logging.getLogger("FFC").setLevel(logging.ERROR) 

In [4]:
import numpy
import pandas
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import sklearn.linear_model as lm
from nltk.corpus import stopwords
import nltk
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn import pipeline,metrics, grid_search 
from sklearn.metrics import f1_score,precision_score
from numpy import genfromtxt
import random

In [20]:
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

with open('sarcasm_dataset.txt','r') as fname:
    file_content = fname.readlines()
random.shuffle(file_content)

output = 'tweets' + "\t" + 'label' + "\n"

for tweet_content in file_content:    
    tweet, label = tweet_content[:-3], tweet_content[-3]
    output += tweet + "\t" + label + "\n"
    
outputfile = open('dataset_csv.csv',"w")
outputfile.write(output)
outputfile.close() 

In [21]:
df = pandas.DataFrame()
train = pandas.DataFrame()
test = pandas.DataFrame()

df = pandas.read_csv("dataset_csv.csv", header=0, sep='\t')

train = df[0:1500]
test = df[1501:]

In [22]:
tfidf_vec = TfidfVectorizer(
            analyzer="word",max_features=None,
            token_pattern=r'\w{1,}',strip_accents='unicode',
            lowercase=True,ngram_range=(1,3),
            min_df=2,use_idf=True,
            smooth_idf=True,norm="l2",
            sublinear_tf=True)

x_train,x_test,y_train,y_test=train_test_split(train['tweets'],train['label'],test_size=0.4,random_state=2,stratify=train['label'])

train_tfidf_matrix=tfidf_vec.fit_transform(x_train)
test_tfidf_matrix=tfidf_vec.fit_transform(x_test)

In [23]:
train = pandas.DataFrame(train_tfidf_matrix.toarray())
test = pandas.DataFrame(test_tfidf_matrix.toarray())

svd = TruncatedSVD(algorithm="randomized", random_state=None, tol=0.0)
scl = StandardScaler()
lr = lm.LogisticRegression(class_weight="balanced", tol = 0.0001)
clf = pipeline.Pipeline([('svd', svd),
    						 ('scl', scl),
                    	     ('lr', lr)])

In [24]:
param_grid = {'svd__n_components' : [250],
                 'svd__n_iter':[1],
                 'lr__C': [11],
                  'lr__penalty':["l1"]}

f_scorer = metrics.make_scorer(f1_score, greater_is_better = True)

model = grid_search.GridSearchCV(estimator = clf, param_grid=param_grid, scoring=f_scorer,
                                     verbose=10, n_jobs=-1, iid=True, refit=True, cv=10)


In [26]:
model.fit(train, y_train)
print("Best score: %0.3f" % model.best_score_)

In [11]:
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
	print("\t%s: %r" % (param_name, best_parameters[param_name]))

Best parameters set:
	lr__C: 11
	lr__penalty: 'l1'
	svd__n_components: 250
	svd__n_iter: 1


In [13]:
best_model = model.best_estimator_
best_model.fit(train,y_train)


Pipeline(steps=[('svd', TruncatedSVD(algorithm='randomized', n_components=250, n_iter=1,
       random_state=None, tol=0.0)), ('scl', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LogisticRegression(C=11, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [ ]:
preds = best_model.predict(test)
preds=list(preds)
target_labels=list(y_test)
print f1_score(target_labels,preds,average="weighted")